In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install arff

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import svm, tree
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_graphviz
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, cross_val_predict, LeaveOneOut
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import random
from datetime import datetime
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

Data reading & preprocessing

In [ ]:
solvent_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/material_project/data/Holistic prediction of enantioselectivity in asymmetric catalysis.Supplementary Data.xlsx', sheet_name = 'solvent')
solvent_data.set_index('solvent_name', inplace = True)
solvent_columns = list(solvent_data.columns)

In [ ]:
nucleophile_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/material_project/data/Holistic prediction of enantioselectivity in asymmetric catalysis.Supplementary Data.xlsx', sheet_name = 'nucleophiles')
nucleophile_data.set_index('nucleophile_name', inplace = True)
nucleophile_columns = list(nucleophile_data.columns)

In [ ]:
raw_catalyst_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/material_project/data/Holistic prediction of enantioselectivity in asymmetric catalysis.Supplementary Data.xlsx', sheet_name = 'full_catalyst')
raw_catalyst_data.rename(columns = {'last catalyst only S used': 'catalyst_name'}, inplace = True)

r_catalyst_data = raw_catalyst_data.copy()
for i in range(17):
    r_catalyst_data.iloc[i, 1] = r_catalyst_data.iloc[i, 1].replace('R/S', 'R')

s_catalyst_data = raw_catalyst_data.copy()
for i in range(17):
    s_catalyst_data.iloc[i, 1] = s_catalyst_data.iloc[i, 1].replace('R/S', 'S')

catalyst_data = pd.concat([r_catalyst_data, s_catalyst_data])
catalyst_data.set_index('catalyst_name', inplace = True)
catalyst_columns = list(catalyst_data.columns)

In [ ]:
raw_catalyst_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/material_project/data/Holistic prediction of enantioselectivity in asymmetric catalysis.Supplementary Data.xlsx', sheet_name = 'full_catalyst')
raw_catalyst_data.rename(columns = {'last catalyst only S used': 'catalyst_name'}, inplace = True)

#Taking care of catalyst R/S
r_catalyst_data = raw_catalyst_data.copy()
for i in range(17):
    r_catalyst_data.iloc[i, 1] = r_catalyst_data.iloc[i, 1].replace('R/S', 'R')
    #print(r_catalyst_data.iloc[i, 1])
#r_catalyst_data.drop(17, inplace = True)

s_catalyst_data = raw_catalyst_data.copy()
for i in range(17):
    s_catalyst_data.iloc[i, 1] = s_catalyst_data.iloc[i, 1].replace('R/S', 'S')
    #print(s_catalyst_data.iloc[i, 1])

catalyst_data = pd.concat([r_catalyst_data, s_catalyst_data])
catalyst_data.rename(columns = {'last catalyst only S used': 'catalyst_name'}, inplace = True)
catalyst_data.set_index('catalyst_name', inplace = True)

catalyst_columns = list(catalyst_data.columns)

# catalyst_data

In [ ]:
iminium_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/material_project/data/Holistic prediction of enantioselectivity in asymmetric catalysis.Supplementary Data.xlsx', sheet_name = 'iminiums')
iminium_data.rename(columns = {'imine': 'iminium_name', 'electronic energy difference (kcal/mol) ': 'electronic energy difference (kcal/mol)'}, inplace = True)
#removed space at the end of electronic energy
iminium_data.set_index('iminium_name', inplace = True)
iminium_data.drop(labels = ['Unnamed: 1'], axis = 1, inplace = True)
for i in range(1, 181):
    z_iminium_name = '(Z)-Iminium ' + str(i)
    e_iminium_name = '(E)-Iminium ' + str(i)
    iminium_data.loc[z_iminium_name, 'electronic energy difference (kcal/mol)'] = iminium_data.loc[e_iminium_name, 'electronic energy difference (kcal/mol)']
iminium_columns = list(iminium_data.columns)

In [ ]:
class Reaction():

    def __init__(self, name, entry, catalyst, nucleophile, substrate, solvent, iminium_type, iminium, majorenantiomer, minorenantiomer, ee, G):
        self.name = name
        self.entry = entry
        self.catalyst = catalyst
        self.nucleophile = nucleophile
        self.substrate = substrate
        self.solvent = solvent
        self.iminium_type = iminium_type
        self.iminium = iminium
        self.majorenantiomer = majorenantiomer
        self.minorenantiomer = minorenantiomer
        self.ee = ee
        self.G = G

        self.solvent_properties = dict()
        for column in solvent_columns:
            self.solvent_properties[column] = solvent_data.loc[solvent, column]

        self.catalyst_properties = dict()
        for column in catalyst_columns:
            self.catalyst_properties[column] = catalyst_data.loc[catalyst, column]

        self.nucleophile_properties = dict()
        for column in nucleophile_columns:
            self.nucleophile_properties[column] = nucleophile_data.loc[nucleophile, column]

        self.e_iminium = '(E)-' + str(iminium)
        self.z_iminium = '(Z)-' + str(iminium)

        self.e_iminium_properties = dict()
        self.z_iminium_properties = dict()
        for column in iminium_columns:
            self.e_iminium_properties[column] = iminium_data.loc[self.e_iminium, column]
            self.z_iminium_properties[column] = iminium_data.loc[self.z_iminium, column]



    def __repr__(self):
        return "Reaction - {}".format(self.name)
        #iminium stuff

In [ ]:
reactions = dict()

def process_data(reaction_number, reaction, iminium_type, sheetname = None):
    reaction_file = '/content/drive/My Drive/Colab Notebooks/material_project/data/reaction information/' + str(reaction_number) + ' ' + reaction + '.xlsx'
    if sheetname == None:
        data = pd.read_excel(reaction_file)
    else:
        data = pd.read_excel(reaction_file, sheet_name = sheetname)

    data.set_index('entry', inplace = True)

    entries = len(data)
    for entry in range(1, entries + 1):
        if sheetname == None:
            reaction_name = reaction + ' ' + str(entry)
        else:
            reaction_name = reaction + ' ' + sheetname + ' ' + str(entry)


        reactions[reaction_name] = Reaction(reaction_name,
                                            entry,
                                            data.loc[entry, 'Catalyst'],
                                            data.loc[entry, 'Nucleophile'],
                                            data.loc[entry, 'Substrate'],
                                            data.loc[entry, 'Solvent'],
                                            iminium_type,
                                            data.loc[entry, 'Iminium'],
                                            data.loc[entry, 'Major Enantiomer'],
                                            data.loc[entry, 'Minor Enantiomer'],
                                            data.loc[entry, 'ee'],
                                            data.loc[entry, 'ΔΔG‡'])



In [ ]:
process_data(1, 'Addition of Alcohols', 'E', 'Scope')
process_data(2, 'Addition of thiols', 'E', 'Catalyst & solvent screening da')
process_data(2, 'Addition of thiols', 'E', 'Effect of catalyst loading')
process_data(2, 'Addition of thiols', 'E', 'Imine scope')
process_data(2, 'Addition of thiols', 'E', 'Thiol scope')
process_data(3, 'Hydrophosphonylation of imines', 'E', 'Catalyst screening data')
process_data(3, 'Hydrophosphonylation of imines', 'E', 'Scope')
process_data(4, 'Addition of diazomethylphosphonates', 'E', "Optimization of catalyst and re")
process_data(4, 'Addition of diazomethylphosphonates', 'E', "Imine scope")
process_data(5, 'Addition of diazoacetamides', 'E', 'Catalyst screening data')
process_data(5, 'Addition of diazoacetamides', 'E', 'Solvent screening data')
process_data(5, 'Addition of diazoacetamides', 'E', 'Substrate(s) scope')
process_data(6, 'Strecker Reaction (with aldimines)', 'E', 'Catalyst screening data')
process_data(6, 'Strecker Reaction (with aldimines)', 'E', 'Solvent screening data')
process_data(6, 'Strecker Reaction (with aldimines)', 'E', 'Imine scope')
process_data(7, 'Peroxidation of imines', 'E', 'Catalyst screening data')
process_data(7, 'Peroxidation of imines', 'E', 'Solvent screening data')
#process_data(7, 'Peroxidation of imines', 'E', 'Substrate(s) scope')
process_data(8, 'Transfer Hydrogenation of b,g-Alkynyl a-Imino Esters', 'E', 'Catalyst screening and reaction')
process_data(8, 'Transfer Hydrogenation of b,g-Alkynyl a-Imino Esters', 'E', 'Scope')
process_data(9, 'Transfer Hydrogenation of Enamides', 'E', 'Scope')
process_data(10, 'Transfer Hydrogenation of N-aryl imines (List)', 'Z', 'Catalyst screening data')
process_data(10, 'Transfer Hydrogenation of N-aryl imines (List)', 'Z', 'Imine scope')
process_data(11, 'Transfer Hydrogenation of N-aryl imines (Rueping)', 'Z', 'Catalyst screening data')
process_data(11, 'Transfer Hydrogenation of N-aryl imines (Rueping)', 'Z', 'Solvent screening data')
process_data(11, 'Transfer Hydrogenation of N-aryl imines (Rueping)', 'Z', 'Imine scope')
#process_data(12, 'Reductive amination of N-aryl imines (Macmillan)', 'Z', 'Reaction optimization')
process_data(13, 'Transfer Hydrogenation of trifluoromethyl ketimines', 'Z', 'Benzothiazoline screening data')
process_data(13, 'Transfer Hydrogenation of trifluoromethyl ketimines', 'Z', 'Imine scope')
process_data(14, 'Transfer Hydrogenation of N-aryl imines by benzothiazoline', 'Z', 'Catalyst screening data')
process_data(14, 'Transfer Hydrogenation of N-aryl imines by benzothiazoline', 'Z', 'Benzothiazoline screening data')
process_data(14, 'Transfer Hydrogenation of N-aryl imines by benzothiazoline', 'Z', 'Imine scope')
#process_data(15, 'Reductive amination of aliphatic ketones by benzothiazoline', 'Z', 'Catalyst screening data')
#process_data(15, 'Reductive amination of aliphatic ketones by benzothiazoline', 'Z', 'Imine scope')
process_data(16, 'Transfer Hydrogenation of ethyl ketimines', 'Z', 'Benzothiazoline screening data')
process_data(16, 'Transfer Hydrogenation of ethyl ketimines', 'Z', 'Scope 1 benzothiazoline')
process_data(16, 'Transfer Hydrogenation of ethyl ketimines', 'Z', 'Scope 2 dihydropyridine')
process_data(17, 'Strecker Reaction (with ketimines)', 'Z', 'Catalyst screening data')
process_data(17, 'Strecker Reaction (with ketimines)', 'Z', 'Solvent data')
process_data(17, 'Strecker Reaction (with ketimines)', 'Z', 'Imine scope')
#process_data(18, 'Addition of enecarbamates to benzoyl imines')
#process_data(19, 'Hydrogenation of fluorinated alkynyl ketimines')
#process_data(20, 'Addition of thiols to imines (Denmark)')

In [ ]:
print(len(reactions))

342


In [ ]:
catalyst = pd.DataFrame(columns = ['Reaction'] + catalyst_columns)
catalyst.set_index('Reaction', inplace = True)

for reaction in reactions:
    for column in catalyst_columns:
        catalyst.loc[reaction, column] = reactions[reaction].catalyst_properties[column]

#first column is numerical

In [ ]:
nucleophile = pd.DataFrame(columns = ['Reaction'] + nucleophile_columns)
nucleophile.set_index('Reaction', inplace = True)

for reaction in reactions:
    for column in nucleophile_columns:
        nucleophile.loc[reaction, column] = reactions[reaction].nucleophile_properties[column]

#first column is numerical

In [ ]:
solvent = pd.DataFrame(columns = ['Reaction'] + solvent_columns)
solvent.set_index('Reaction', inplace = True)

for reaction in reactions:
    for column in solvent_columns:
        solvent.loc[reaction, column] = reactions[reaction].solvent_properties[column]

#first column is numerical

In [ ]:
iminium = pd.DataFrame(columns = ['Reaction', "iminium_type"] + iminium_columns)
iminium.set_index('Reaction', inplace = True)

for reaction in reactions:
    if reactions[reaction].iminium_type == 'E':
        iminium.loc[reaction, "iminium_type"] = 'E'
        for column in iminium_columns:
            iminium.loc[reaction, column] = reactions[reaction].e_iminium_properties[column]
    elif reactions[reaction].iminium_type == 'Z':
        iminium.loc[reaction, "iminium_type"] = 'Z'
        for column in iminium_columns:
            iminium.loc[reaction, column] = reactions[reaction].z_iminium_properties[column]



In [ ]:
ee = pd.DataFrame(columns = ['Reaction', 'ee'])
ee.set_index('Reaction', inplace = True)

for reaction in reactions:
    ee.loc[reaction, 'ee'] = reactions[reaction].ee

In [ ]:
Y = pd.DataFrame(columns = ['Reaction', 'ΔΔG‡'])
Y.set_index('Reaction', inplace = True)

for reaction in reactions:
    Y.loc[reaction, 'ΔΔG‡'] = reactions[reaction].G


In [ ]:
print(catalyst.shape, nucleophile.shape, solvent.shape, iminium.shape, ee.shape)

(342, 85) (342, 15) (342, 160) (342, 22) (342, 1)


In [ ]:
X_iminium = pd.concat([catalyst.drop(['Ar group'], axis = 1),
               nucleophile.drop(['nucleophile'], axis = 1),
               solvent.drop(['solvent'], axis = 1),
               iminium.drop(['iminium_type'], axis = 1)], axis = 1)
X_iminium.shape

(342, 278)

In [ ]:
#no iminium features
X_no_iminium = pd.concat([catalyst.drop(['Ar group'], axis = 1),
               nucleophile.drop(['nucleophile'], axis = 1),
               solvent.drop(['solvent'], axis = 1)], axis = 1)
X_no_iminium.shape

(342, 257)

In [ ]:
X_no_nucleophile = pd.concat([catalyst.drop(['Ar group'], axis = 1),
                   solvent.drop(['solvent'], axis = 1)], axis = 1)
X_no_nucleophile.shape

(342, 243)

In [ ]:
for reaction in reactions:
    if iminium.loc[reaction, 'iminium_type'] == 'Z':
        Y.loc[reaction, 'ΔΔG‡'] = Y.loc[reaction, 'ΔΔG‡'] * (-1)

Run 100 replications with Monte-carlo cross validation

In [ ]:
X = X_iminium

In [ ]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X, Y, test_size = 0.1, random_state = 1)

Bayes opt for SVR parameter C


In [ ]:
import os
import pandas as pd
import numpy as np
import sklearn
import random
import heapq
import matplotlib.pyplot as plt


from numpy import vstack
from numpy import argmax
from numpy import asarray
from numpy.random import normal

from scipy.stats import norm
from warnings import catch_warnings
from warnings import simplefilter
from matplotlib import pyplot


from scipy import stats
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import pearsonr
from numpy.random import random


from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
X.shape

(342, 278)

In [ ]:

# X_train, X_test, Y_train, Y_test = train_test_split( X_train1,  Y_train1, test_size = 0.2, random_state = 100)
X_train_train, X_valid, Y_train_train, Y_valid = train_test_split( X_train1,Y_train1, test_size = 0.2, random_state = 100)



X_valid=X_valid.to_numpy()
Y_valid=Y_valid.to_numpy()
X_train_train=X_train_train.to_numpy()
Y_train_train=Y_train_train.to_numpy()

Bayesian optimization for hyperparameter tuning

In [ ]:

validation_replication=1
result_valid=np.zeros(validation_replication)
result_valid=np.reshape(result_valid,(1,validation_replication))








def objective(x1):

    # Constraints for each hyperparameter
    # All hyperparameters are positive integers.
    # Minimum of "min_samples_split" : 2.

    a1 = int(x1)


    if a1<2:
        a1=2


# Performance evaluation for  validation set
    for j in range(0,1):
        # Fit the model with training data and check the Mean Absolute Percentage Error (MSE) of validation data
        reg_m = make_pipeline(StandardScaler(), DecisionTreeRegressor(min_samples_split=a1,random_state=100))
        reg_m.fit(X_train_train, Y_train_train.ravel());

        predicted_y_m = reg_m.predict(X_valid);
        prediction=np.reshape(predicted_y_m,(predicted_y_m.shape[0],1))
        result_valid[0,j] =(mean_squared_error(Y_valid, prediction))
    # Since Bayesian Optimization is maximization problem, we used reciprocal as output.
    return (1/np.mean(result_valid))

# surrogate or approximation for the objective function
def surrogate(model, X):
    # catch any warning generated when making a prediction
    #with catch_warnings():
        # ignore generated warnings if the distribution is thin at a given point
        #simplefilter("ignore")
        return model.predict(X, return_std=True)

# We used Expected Improvement as our acquisition function.
def acquisition(X, Xsamples, model):
    # calculate the best surrogate score found so far
    yhat, _ = surrogate(model, X)
    best = max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)

    #mu = mu[:, 0]
    # calculate the Expected improvement
    probs = (mu - best) * norm.cdf((mu - best) / (std + 1E-9)) + (std + 1E-9) * norm.pdf((mu - best) / (std + 1E-9))
    return probs

# optimize the acquisition function
def opt_acquisition(X, y, model):
    np.random.seed(i*100+h*100+100*i1)
    # random search, generate random samples
    X1samples = 10 * random((100,1))

    # calculate the acquisition function for each sample
    Xsamples = X1samples
    scores = acquisition(X, Xsamples, model)
    # locate the index of the largest scores
    ix = argmax(scores)
    return Xsamples[ix,]



X1 = [2]
X1 = asarray(X1)


y = asarray([objective(X1)])
X = X1
X = X.reshape(len(X1),1)
y = y.reshape(len(y),1)
h=0




# Starting from given 20 points
for i1 in range(0,20):
    np.random.seed(i1*100+i*100)
    X1 = 10* random(1)

    ysample = asarray([objective(X1)])
    Xsample = X1
    Xsample = Xsample.reshape(len(X1),1)
    ysample = ysample.reshape(len(ysample),1)

    X = vstack((X,Xsample))
    y = vstack((y,ysample))
# define the surrogate model
model = GaussianProcessRegressor()
model.fit(X, y)



# Sample new points (hyperparameters) with Bayesian Optimization.
# It sequentialy samples 100 points based on the optimization.
for h in range(100):
    print(h)
    # select the next point to sample
    x = opt_acquisition(X, y, model)
    x = asarray(x)
    # sample the point
    actual = objective(x[0])
    est, _ = surrogate(model,[x])
    actual = asarray(actual)
    # add the data to the dataset
    X = vstack((X,[x]))
    y = vstack((y,[[actual]]))
    # update the model
    model.fit(X, y)
    # best result
    ix = argmax(y)
    print("Current")
    print(x[0],1/est,1/actual)

#Provide Best hyperparameters settings based on Bayesian Optimization.
print("Best")
print((X[ix, 0],y[ix],1/y[ix]),ix)



a1=int(X[ix,0])

if a1<2:
    a1=2
print(a1)



<ipython-input-29-2f337655382e>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a1 = int(x1)
<ipython-input-29-2f337655382e>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a1 = int(x1)
<ipython-input-29-2f337655382e>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a1 = int(x1)
<ipython-input-29-2f337655382e>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element fr

0
Current
3.902529325676994 [0.03497818] 0.3309186575451189
1
Current
4.260061159549615 [0.12022807] 0.304034465887485
2
Current
3.3041405619364896 [0.07600708] 0.3309186575451189
3
Current
4.458188215517102 [0.25284539] 0.304034465887485
4
Current
0.9578699001717861 [0.25859949] 0.4801814333215458
5
Current
0.026649159002452016 [0.23806992] 0.4801814333215458
6
Current
8.822173801442403 [0.24369047] 0.5056607628429768
7
Current
4.541313815866768 [0.2918932] 0.304034465887485
8
Current
4.644787287349102 [0.29742018] 0.304034465887485
9
Current
4.629466477777406 [0.29960075] 0.304034465887485
10
Current
4.60184367480022 [0.30061208] 0.304034465887485
11
Current
0.4617290721566225 [0.47942173] 0.4801814333215458
12
Current
3.3488592016866536 [0.33478457] 0.3309186575451189
13
Current
4.604069541526378 [0.30123073] 0.304034465887485
14
Current
8.090267093236832 [0.50727133] 0.5056607628429768
15
Current
4.5994632125340384 [0.30170831] 0.304034465887485
16
Current
9.313199526042533 [0.8203

In [ ]:
a1


4

Run 100 replications with Monte-carlo cross validation Calculate mean & standard deviation of MSE and R^2

In [ ]:
def run_random_forest(iterations,value):
    scores = pd.DataFrame(columns = ['iteration', 'MSE', 'test r^2', 'train r^2', 'total r^2'])
    #scores.set_index('iteration', inplace = True)
    for i in range(iterations):
        a=10*i
        parameter=a1
        forest = make_pipeline(StandardScaler(), DecisionTreeRegressor(min_samples_split=a1,random_state=100))
        X_train, X_test, Y_train, Y_test = train_test_split(X_train1,  Y_train1, test_size = 0.2, random_state = a)

        forest.fit(X_train.reset_index().drop(['Reaction'], axis = 1), Y_train.reset_index().drop('Reaction', axis = 1).values.ravel())

        #evaluating performance
        Y_pred = forest.predict(X_test.reset_index().drop(['Reaction'], axis = 1))

        results = pd.concat([Y_test.reset_index(), pd.DataFrame(Y_pred)], axis = 1)
        results.set_index('Reaction', inplace = True)
        results.columns = ['Actual', 'Predicted']

        Y_train_pred = forest.predict(X_train.reset_index().drop(['Reaction'], axis = 1))
        train_results = pd.concat([Y_train.reset_index(), pd.DataFrame(Y_train_pred)], axis = 1)
        train_results.set_index('Reaction', inplace = True)
        train_results.columns = ['Actual', 'Predicted']

        all_results = pd.concat([train_results, results])

        scores = scores.append({'iteration': str(i+1),
                    'MSE': mean_squared_error(Y_test, Y_pred),
                    'test r^2': r2_score(Y_test, Y_pred),
                    'train r^2': r2_score(train_results['Actual'], train_results['Predicted']),
                    'total r^2': r2_score(all_results['Actual'], all_results['Predicted'])}, ignore_index = True)

    return scores

In [ ]:
import warnings
warnings.filterwarnings("ignore")
for i in range(1):
  print("Parameter")
  print(20*i)
  results_rf = run_random_forest(100,i)
  print(results_rf)
  print(results_rf.mean())
  print(results_rf.std())

Parameter
0
   iteration       MSE  test r^2  train r^2  total r^2
0          1  0.255967  0.903034   0.994494   0.977629
1          2  0.273373  0.912697   0.991916   0.974598
2          3  0.327078  0.882599   0.995426   0.973481
3          4  0.367716  0.856592   0.993045   0.968589
4          5  0.193520  0.934367   0.992215   0.980290
..       ...       ...       ...        ...        ...
95        96  0.133122  0.959186   0.992379   0.984829
96        97  0.303154  0.911008   0.993665   0.974055
97        98  0.325646  0.885122   0.993429   0.971963
98        99  0.244757  0.914273   0.994405   0.978433
99       100  0.193082  0.915572   0.993466   0.981034

[100 rows x 5 columns]
iteration    1.234568e+189
MSE           3.380920e-01
test r^2      8.798194e-01
train r^2     9.933809e-01
total r^2     9.710895e-01
dtype: float64
MSE          0.146178
test r^2     0.055837
train r^2    0.001286
total r^2    0.010317
dtype: float64


Analysis without Imine's features

In [ ]:
X = X_iminium

In [ ]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X, Y, test_size = 0.1, random_state = 1)

In [ ]:
os.chdir('/content/drive/My Drive/Colab Notebooks/material_project/code/original/chem_brandeis/IEEE_BIBM')
X_train2 = pd.read_excel('X_train2.xlsx', index_col=0)

In [ ]:
print(X_train2.shape)
print(X_no_iminium.shape)

(307, 257)
(342, 257)


In [ ]:
import os
import pandas as pd
import numpy as np
import sklearn
import random
import heapq
import matplotlib.pyplot as plt


from numpy import vstack
from numpy import argmax
from numpy import asarray
from numpy.random import normal

from scipy.stats import norm
from warnings import catch_warnings
from warnings import simplefilter
from matplotlib import pyplot


from scipy import stats
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import pearsonr
from numpy.random import random


from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

Bayesian optimization for hyperparameter tuning

In [ ]:

X_train_train, X_valid, Y_train_train, Y_valid = train_test_split( X_train2,Y_train1, test_size = 0.2, random_state = 100)



X_valid=X_valid.to_numpy()
Y_valid=Y_valid.to_numpy()
X_train_train=X_train_train.to_numpy()
Y_train_train=Y_train_train.to_numpy()

In [ ]:

validation_replication=1
result_valid=np.zeros(validation_replication)
result_valid=np.reshape(result_valid,(1,validation_replication))








def objective(x1):

    # Constraints for each hyperparameter
    # All hyperparameters are positive integers.
    # Minimum of "min_samples_split" : 2.

    a1 = int(x1)


    if a1<2:
        a1=2


# Performance evaluation for  validation set
    for j in range(0,1):
        # Fit the model with training data and check the Mean Absolute Percentage Error (MSE) of validation data
        reg_m = make_pipeline(StandardScaler(), DecisionTreeRegressor(min_samples_split=a1,random_state=100))
        reg_m.fit(X_train_train, Y_train_train.ravel());

        predicted_y_m = reg_m.predict(X_valid);
        prediction=np.reshape(predicted_y_m,(predicted_y_m.shape[0],1))
        result_valid[0,j] =(mean_squared_error(Y_valid, prediction))
    # Since Bayesian Optimization is maximization problem, we used reciprocal as output.
    return (1/np.mean(result_valid))

# surrogate or approximation for the objective function
def surrogate(model, X):
    # catch any warning generated when making a prediction
    #with catch_warnings():
        # ignore generated warnings if the distribution is thin at a given point
        #simplefilter("ignore")
        return model.predict(X, return_std=True)

# We used Expected Improvement as our acquisition function.
def acquisition(X, Xsamples, model):
    # calculate the best surrogate score found so far
    yhat, _ = surrogate(model, X)
    best = max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)

    #mu = mu[:, 0]
    # calculate the Expected improvement
    probs = (mu - best) * norm.cdf((mu - best) / (std + 1E-9)) + (std + 1E-9) * norm.pdf((mu - best) / (std + 1E-9))
    return probs

# optimize the acquisition function
def opt_acquisition(X, y, model):
    np.random.seed(i*100+h*100+100*i1)
    # random search, generate random samples
    X1samples = 10 * random((100,1))

    # calculate the acquisition function for each sample
    Xsamples = X1samples
    scores = acquisition(X, Xsamples, model)
    # locate the index of the largest scores
    ix = argmax(scores)
    return Xsamples[ix,]



X1 = [2]
X1 = asarray(X1)


y = asarray([objective(X1)])
X = X1
X = X.reshape(len(X1),1)
y = y.reshape(len(y),1)
h=0




# Starting from given 20 points
for i1 in range(0,20):
    np.random.seed(i1*100+i*100)
    X1 = 10* random(1)

    ysample = asarray([objective(X1)])
    Xsample = X1
    Xsample = Xsample.reshape(len(X1),1)
    ysample = ysample.reshape(len(ysample),1)

    X = vstack((X,Xsample))
    y = vstack((y,ysample))
# define the surrogate model
model = GaussianProcessRegressor()
model.fit(X, y)



# Sample new points (hyperparameters) with Bayesian Optimization.
# It sequentialy samples 100 points based on the optimization.
for h in range(100):
    print(h)
    # select the next point to sample
    x = opt_acquisition(X, y, model)
    x = asarray(x)
    # sample the point
    actual = objective(x[0])
    est, _ = surrogate(model,[x])
    actual = asarray(actual)
    # add the data to the dataset
    X = vstack((X,[x]))
    y = vstack((y,[[actual]]))
    # update the model
    model.fit(X, y)
    # best result
    ix = argmax(y)
    print("Current")
    print(x[0],1/est,1/actual)

#Provide Best hyperparameters settings based on Bayesian Optimization.
print("Best")
print((X[ix, 0],y[ix],1/y[ix]),ix)



a1=int(X[ix,0])

if a1<2:
    a1=2
print(a1)



0
Current
8.871119202964982 [0.00483984] 0.1543091713431736
1
Current
8.464963654587647 [0.05035257] 0.1543091713431736
2
Current
7.471871555035737 [0.12900825] 0.21265141045778982
3
Current
9.31379414084753 [0.08409581] 0.15754253715423086
4
Current
0.4930579350443631 [0.10557484] 0.20384555220700945
5
Current
0.03658394151639821 [0.06405421] 0.20384555220700945
6
Current
9.747659276927493 [0.11934913] 0.15754253715423086
7
Current
8.306448951298757 [0.14730501] 0.1543091713431736
8
Current
9.922428909218976 [0.1493968] 0.15754253715423086
9
Current
8.194040363601033 [0.1502365] 0.1543091713431736
10
Current
8.459939742093642 [0.15626334] 0.1543091713431736
11
Current
5.107935611666639 [0.23133409] 0.2103837047352026
12
Current
9.520526598576012 [0.15717619] 0.15754253715423086
13
Current
8.485480859046284 [0.15572636] 0.1543091713431736
14
Current
6.495429861057857 [0.17062198] 0.17110438623027918
15
Current
8.205906733072887 [0.15204461] 0.1543091713431736
16
Current
6.0958762937404

In [ ]:
a1


8

Run 100 replications with Monte-carlo cross validation Calculate mean & standard deviation of MSE and R^2

In [ ]:
def run_random_forest(iterations,value):
    scores = pd.DataFrame(columns = ['iteration', 'MSE', 'test r^2', 'train r^2', 'total r^2'])
    #scores.set_index('iteration', inplace = True)
    for i in range(iterations):
        a=10*i
        parameter=a1
        forest = make_pipeline(StandardScaler(), DecisionTreeRegressor(min_samples_split=a1,random_state=100))
        X_train, X_test, Y_train, Y_test = train_test_split(X_train2,  Y_train1, test_size = 0.2, random_state = a)

        forest.fit(X_train.reset_index().drop(['Reaction'], axis = 1), Y_train.reset_index().drop('Reaction', axis = 1).values.ravel())

        #evaluating performance
        Y_pred = forest.predict(X_test.reset_index().drop(['Reaction'], axis = 1))

        results = pd.concat([Y_test.reset_index(), pd.DataFrame(Y_pred)], axis = 1)
        results.set_index('Reaction', inplace = True)
        results.columns = ['Actual', 'Predicted']

        Y_train_pred = forest.predict(X_train.reset_index().drop(['Reaction'], axis = 1))
        train_results = pd.concat([Y_train.reset_index(), pd.DataFrame(Y_train_pred)], axis = 1)
        train_results.set_index('Reaction', inplace = True)
        train_results.columns = ['Actual', 'Predicted']

        all_results = pd.concat([train_results, results])

        scores = scores.append({'iteration': str(i+1),
                    'MSE': mean_squared_error(Y_test, Y_pred),
                    'test r^2': r2_score(Y_test, Y_pred),
                    'train r^2': r2_score(train_results['Actual'], train_results['Predicted']),
                    'total r^2': r2_score(all_results['Actual'], all_results['Predicted'])}, ignore_index = True)

    return scores

In [ ]:
import warnings
warnings.filterwarnings("ignore")
for i in range(1):
  print("Parameter")
  print(20*i)
  results_rf = run_random_forest(100,i)
  print(results_rf)
  print(results_rf.mean())
  print(results_rf.std())

Parameter
0
   iteration       MSE  test r^2  train r^2  total r^2
0          1  0.242138  0.908272   0.975950   0.963534
1          2  0.123460  0.960572   0.967414   0.966026
2          3  0.263884  0.905282   0.972416   0.959543
3          4  0.260854  0.898267   0.969217   0.956541
4          5  0.201271  0.931739   0.968691   0.961082
..       ...       ...       ...        ...        ...
95        96  0.173771  0.946723   0.968856   0.963894
96        97  0.759595  0.777017   0.969234   0.923830
97        98  0.256832  0.909398   0.974045   0.961239
98        99  0.196753  0.931086   0.974123   0.965653
99       100  0.319039  0.860495   0.969890   0.952505

[100 rows x 5 columns]
iteration    1.234568e+189
MSE           2.595422e-01
test r^2      9.081516e-01
train r^2     9.705540e-01
total r^2     9.584072e-01
dtype: float64
MSE          0.106726
test r^2     0.038225
train r^2    0.003277
total r^2    0.007649
dtype: float64
